<a href="https://colab.research.google.com/github/JorgeSedek/tp2_orga_datos/blob/main/TP2_S18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=94c0b7e8d9780bd2a432464def4957306a44a91019c3c60942fa42a6f2d85e8a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/com

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
rdd_movies = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/movies.parquet').rdd
rdd_keywords = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/keywords.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


S18 (). ¿Cuál es el género con más películas cada década? ¿Y la keyword para cada década?

In [ ]:
rdd_keywords.take(5)

[Row(id=862, keywords='jealousy,toy,boy,friendship,friends,rivalry,boy next door,new toy,toy comes to life'),
 Row(id=8844, keywords="board game,disappearance,based on children's book,new home,recluse,giant insect"),
 Row(id=15602, keywords='fishing,best friend,duringcreditsstinger,old men'),
 Row(id=31357, keywords='based on novel,interracial relationship,single mother,divorce,chick flick'),
 Row(id=11862, keywords='baby,midlife crisis,confidence,aging,daughter,mother daughter relationship,pregnancy,contraception,gynecologist')]

In [ ]:
rdd_movies.first()

Row(adult='False', belongs_to_collection='Toy Story Collection', budget='30000000', genres='Animation,Comedy,Family', homepage='http://toystory.disney.com/toy-story', id='862', imdb_id='tt0114709', original_language='en', original_title='Toy Story', overview="Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", popularity=21.946943, poster_path='/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', production_companies='3', production_countries="[{'iso_3166_1': 'US', 'name': 'United States of America'}]", release_date='1995-10-30', revenue=373554033.0, runtime=81.0, spoken_languages="[{'iso_639_1': 'en', 'name': 'English'}]", status='Released', tagline=None, title='Toy Story', video=False, vote_average=7.7, vote_count=5415.0)

In [ ]:
rdd_filtrado = rdd_movies.filter(lambda x: not pd.isnull(x.genres) and (not pd.isnull(x.release_date)))

In [ ]:
tabla_prueba = rdd_filtrado.map(lambda x: (x.id,x.genres,x.release_date))

In [ ]:
tabla_prueba2 = tabla_prueba.map(lambda x: (x[0], x[1].split(","), x[2].split("-")[0][:-1] + '0'))
tabla_prueba2.take(5)

[('862', ['Animation', 'Comedy', 'Family'], '1990'),
 ('8844', ['Adventure', 'Fantasy', 'Family'], '1990'),
 ('15602', ['Romance', 'Comedy'], '1990'),
 ('31357', ['Comedy', 'Drama', 'Romance'], '1990'),
 ('11862', ['Comedy'], '1990')]

In [ ]:
tabla_prueba3 = tabla_prueba2.flatMap(lambda x:  (((x[2], x[1][i]), 1) for i in range(len(x[1]))))
tabla_prueba3.take(5)

[(('1990', 'Animation'), 1),
 (('1990', 'Comedy'), 1),
 (('1990', 'Family'), 1),
 (('1990', 'Adventure'), 1),
 (('1990', 'Fantasy'), 1)]

In [ ]:
tabla_prueba4 = tabla_prueba3.filter(lambda x: not pd.isnull(x[0]))
tabla_prueba5 = tabla_prueba4.reduceByKey(lambda a,b: a+b)
tabla_prueba5.take(10)

[(('1990', 'Family'), 485),
 (('1990', 'Fantasy'), 362),
 (('1990', 'Romance'), 995),
 (('1990', 'Drama'), 2613),
 (('1990', 'Action'), 995),
 (('1990', 'Thriller'), 1049),
 (('1990', 'Horror'), 465),
 (('1990', 'Science Fiction'), 482),
 (('1990', 'Documentary'), 292),
 (('1970', 'Drama'), 1508)]

In [ ]:
tabla_prueba6 = tabla_prueba5.map(lambda x: (x[0][0], (x[0][1], x[1])))
tabla_prueba6.take(5)

[('1990', ('Family', 485)),
 ('1990', ('Fantasy', 362)),
 ('1990', ('Romance', 995)),
 ('1990', ('Drama', 2613)),
 ('1990', ('Action', 995))]

In [ ]:
tabla_prueba7 = tabla_prueba6.reduceByKey(lambda a,b: a if a[1] > b[1] else b)
tabla_prueba7.takeOrdered(1000, key=lambda x: x[0])                                                                                        

[('1870', ('Documentary', 2)),
 ('1880', ('Documentary', 4)),
 ('1890', ('Documentary', 28)),
 ('1900', ('Fantasy', 30)),
 ('1910', ('Drama', 81)),
 ('1920', ('Drama', 257)),
 ('1930', ('Drama', 707)),
 ('1940', ('Drama', 781)),
 ('1950', ('Drama', 1065)),
 ('1960', ('Drama', 1212)),
 ('1970', ('Drama', 1508)),
 ('1980', ('Drama', 1553)),
 ('1990', ('Drama', 2613)),
 ('2000', ('Drama', 5282)),
 ('2010', ('Drama', 5183)),
 ('2020', ('Adventure', 1))]

In [ ]:
tabla_prueba7.map(lambda x: (x[0], x[1][0])).collect()

[('1990', 'Drama'),
 ('1970', 'Drama'),
 ('1980', 'Drama'),
 ('1940', 'Drama'),
 ('2010', 'Drama'),
 ('1920', 'Drama'),
 ('1890', 'Documentary'),
 ('2020', 'Adventure'),
 ('1880', 'Documentary'),
 ('1960', 'Drama'),
 ('1930', 'Drama'),
 ('2000', 'Drama'),
 ('1950', 'Drama'),
 ('1910', 'Drama'),
 ('1900', 'Fantasy'),
 ('1870', 'Documentary')]

¿Y la keyword para cada década?

In [ ]:
rdd_keywords.take(5)

[Row(id=862, keywords='jealousy,toy,boy,friendship,friends,rivalry,boy next door,new toy,toy comes to life'),
 Row(id=8844, keywords="board game,disappearance,based on children's book,new home,recluse,giant insect"),
 Row(id=15602, keywords='fishing,best friend,duringcreditsstinger,old men'),
 Row(id=31357, keywords='based on novel,interracial relationship,single mother,divorce,chick flick'),
 Row(id=11862, keywords='baby,midlife crisis,confidence,aging,daughter,mother daughter relationship,pregnancy,contraception,gynecologist')]

In [ ]:
rdd_keywords2 = rdd_keywords.filter(lambda x: not pd.isnull(x.id) and (not pd.isnull(x.keywords)))

In [ ]:
rdd_keywords2.first()

Row(id=862, keywords='jealousy,toy,boy,friendship,friends,rivalry,boy next door,new toy,toy comes to life')

In [ ]:
rdd_keywords3 = rdd_keywords2.map(lambda x: (x[0], x[1].replace(","," ").split()))

In [ ]:
rdd_keywords3.first()

(862,
 ['jealousy',
  'toy',
  'boy',
  'friendship',
  'friends',
  'rivalry',
  'boy',
  'next',
  'door',
  'new',
  'toy',
  'toy',
  'comes',
  'to',
  'life'])

In [ ]:
rdd_filtrado_fecha = rdd_movies.filter(lambda x: not pd.isnull(x.release_date) and (not "-" in x.id))

In [ ]:
rdd_decada = rdd_filtrado_fecha.map(lambda x: (int(x.id),x.release_date)).map(lambda x: (x[0], x[1].split("-")[0][:-1] + '0'))

In [ ]:
rdd_fecha_keyword = rdd_keywords3.join(rdd_decada)

In [ ]:
rdd_fecha_keyword2 = rdd_fecha_keyword.map(lambda x: (x[1][1], x[1][0]))

In [ ]:
rdd_decada_keyword = rdd_fecha_keyword2.reduceByKey(lambda a, b: a + b)

In [ ]:
import collections

In [ ]:
rdd_decada_keyword.map(lambda x: (x[0], sorted(collections.Counter(x[1]) , key=collections.Counter(x[1]).get, reverse=True)[0])).collect()

[('1980', 'nudity'),
 ('1920', 'film'),
 ('1970', 'nudity'),
 ('1910', 'film'),
 ('1870', 'silent'),
 ('1990', 'film'),
 ('1950', 'film'),
 ('2010', 'woman'),
 ('1900', 'film'),
 ('2020', 'sequel'),
 ('1960', 'film'),
 ('1930', 'musical'),
 ('1940', 'noir'),
 ('2000', 'film'),
 ('1890', 'film'),
 ('1880', 'short')]